In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import sqlite3

In [8]:
# found a table containing longitude and latitudes of countries and 
# brought it into pandas

long_lat=pd.read_html("https://developers.google.com/public-data/docs/canonical/countries_csv",skiprows=1)[0]

# renaming columns
long_lat.columns = ['country_code', 'latitude','longitude','name']

# printing dataframe
long_lat.head()

,country_code,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [9]:
#  country code 
converting_country_code=pd.read_html("https://www.worldatlas.com/aatlas/ctycodes.htm",skiprows=1)[0]
converting_country_code.columns= ['name','country_code_2','country_code_3','x','y']
converting_country_code=converting_country_code.drop(['x', 'y'], axis=1)
converting_country_code.head()


,name,country_code_2,country_code_3
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND


In [10]:
merged_col=pd.merge(converting_country_code, long_lat, left_on='country_code_2',right_on="country_code",how="inner")
merged_col.head()



,name_x,country_code_2,country_code_3,country_code,latitude,longitude,name_y
0,Afghanistan,AF,AFG,AF,33.939110,67.709953,Afghanistan
1,Albania,AL,ALB,AL,41.153332,20.168331,Albania
2,Algeria,DZ,DZA,DZ,28.033886,1.659626,Algeria
3,American Samoa,AS,ASM,AS,-14.270972,-170.132217,American Samoa
4,Andorra,AD,AND,AD,42.546245,1.601554,Andorra


In [11]:
# cleaning up the pandas dataframe
# dropping cols I dont need
merged_col=merged_col.drop(['name_y','country_code','country_code_2'], axis=1)
merged_col.head()


,name_x,country_code_3,latitude,longitude
0,Afghanistan,AFG,33.939110,67.709953
1,Albania,ALB,41.153332,20.168331
2,Algeria,DZA,28.033886,1.659626
3,American Samoa,ASM,-14.270972,-170.132217
4,Andorra,AND,42.546245,1.601554


In [12]:
# renaming cols
merged_col.columns = ['country','country_code','latitude','longitude']

In [13]:
merged_col.head()


,country,country_code,latitude,longitude
0,Afghanistan,AFG,33.939110,67.709953
1,Albania,ALB,41.153332,20.168331
2,Algeria,DZA,28.033886,1.659626
3,American Samoa,ASM,-14.270972,-170.132217
4,Andorra,AND,42.546245,1.601554


In [14]:
# Saved the merged cols into a csv
merged_col.to_csv("/Users/muhammadwaliji/Desktop/project_2/country_code.csv")


In [15]:
# Used the WHO API to get access to how they add the country code and will be 
# merging it with our own table to make sure the data we intend to parse in future from WHO alligns with our 
# merged_col
url = "http://apps.who.int/gho/athena/api/COUNTRY?format=json"
country_r = requests.get(url).json()
country_r


{'copyright': '(c) World Health Organization',
 'dataset': [],
 'attribute': [{'label': 'DS', 'display': 'DS'},
  {'label': 'FIPS', 'display': 'FIPS'},
  {'label': 'IOC', 'display': 'IOC'},
  {'label': 'ISO2', 'display': 'ISO2'},
  {'label': 'ISO', 'display': 'ISO'},
  {'label': 'ITU', 'display': 'ITU'},
  {'label': 'MARC', 'display': 'MARC'},
  {'label': 'WHO', 'display': 'WHO'},
  {'label': 'WMO', 'display': 'WMO'},
  {'label': 'GEOMETRY', 'display': 'GEOMETRY'},
  {'label': 'MORT', 'display': 'MORT'},
  {'label': 'LAND_AREA_KMSQ_2012', 'display': 'LAND_AREA_KMSQ_2012'},
  {'label': 'LANGUAGES_EN_2012', 'display': 'LANGUAGES_EN_2012'},
  {'label': 'WHO_REGION', 'display': 'WHO_REGION'},
  {'label': 'WHO_REGION_CODE', 'display': 'WHO_REGION_CODE'},
  {'label': 'WORLD_BANK_INCOME_GROUP', 'display': 'World Bank income group'},
  {'label': 'WORLD_BANK_INCOME_GROUP_CODE',
   'display': 'World Bank income group code'},
  {'label': 'SHORTNAMEES', 'display': 'SHORTNAMEES'},
  {'label': 'SHOR

In [16]:
# checking to see what needs to be added into the country_r to output the country_code 

country_r['dimension'][0]['code'][0]['label']


'AFG'

In [17]:
# Creating a list of all the country_codes I can get from the WHO website

who_country_list=[]

country_code=country_r['dimension'][0]['code']

for country in country_code:
    who_country_list.append(country["label"])

In [18]:
# converting the list into a dataframe and renaming the col. I did this 
# so I can merge this dataframe with the one we made earlier to make sure all the who countries are in the mergel_col

who_df=pd.DataFrame(who_country_list)
who_df.columns=["who_country"]
who_df.head()


,who_country
0,AFG
1,ALB
2,DZA
3,AND
4,AGO


In [19]:
# merging the two data frames together on country code. this output 226 
# countries which is sufficent for our data set
who_and_others=pd.merge(merged_col, who_df, left_on='country_code',right_on="who_country",how="inner")
who_and_others.head()


,country,country_code,latitude,longitude,who_country
0,Afghanistan,AFG,33.939110,67.709953,AFG
1,Albania,ALB,41.153332,20.168331,ALB
2,Algeria,DZA,28.033886,1.659626,DZA
3,American Samoa,ASM,-14.270972,-170.132217,ASM
4,Andorra,AND,42.546245,1.601554,AND


In [20]:
who_and_others=who_and_others.drop(['who_country'], axis=1)


In [21]:
who_and_others.head(1)

,country,country_code,latitude,longitude
0,Afghanistan,AFG,33.93911,67.709953


In [22]:
who_and_others.to_csv("/Users/muhammadwaliji/Desktop/project_2/who_inc_country_code.csv")



In [23]:
# Option 1 use pd.to_sql
# Option 2 write insert statements in the code  

## Option 1: Use pd.to_sql

In [24]:
sqlite_filename = '/Users/muhammadwaliji/Desktop/project_2/who_country_code_mhw.sqlite'

In [25]:
# Need to have a database with many tables one of which is this one
# but having either a table named this or a database named this but
# not both


# sqlite_filename = '/Users/muhammadwaliji/Desktop/project_2/who_country_code_mhw.db'



In [26]:
conn = sqlite3.connect(sqlite_filename)

who_and_others.to_sql("long_lat_frompandas", conn, if_exists="replace")

# conn.close()

In [27]:
cur = conn.cursor()
cur.execute("SELECT * FROM long_lat_frompandas")
rows = cur.fetchall()
for row in rows:
    print(row)


(0, 'Afghanistan', 'AFG', 33.93911, 67.709953)
(1, 'Albania', 'ALB', 41.153332, 20.168331)
(2, 'Algeria', 'DZA', 28.033886, 1.6596259999999998)
(3, 'American Samoa', 'ASM', -14.270972, -170.132217)
(4, 'Andorra', 'AND', 42.546245, 1.6015540000000001)
(5, 'Angola', 'AGO', -11.202691999999999, 17.873887)
(6, 'Anguilla', 'AIA', 18.220554, -63.068615)
(7, 'Antigua and Barbuda', 'ATG', 17.060816, -61.796428000000006)
(8, 'Argentina', 'ARG', -38.416097, -63.616671999999994)
(9, 'Armenia', 'ARM', 40.069099, 45.038189)
(10, 'Aruba', 'ABW', 12.52111, -69.968338)
(11, 'Australia', 'AUS', -25.274398, 133.775136)
(12, 'Austria', 'AUT', 47.516231, 14.550072)
(13, 'Azerbaijan', 'AZE', 40.143105, 47.576927000000005)
(14, 'Bahamas', 'BHS', 25.03428, -77.39628)
(15, 'Bahrain', 'BHR', 25.930414000000003, 50.637772)
(16, 'Bangladesh', 'BGD', 23.684994, 90.35633100000001)
(17, 'Barbados', 'BRB', 13.193887, -59.543198)
(18, 'Belarus', 'BLR', 53.709807, 27.953389)
(19, 'Belgium', 'BEL', 50.503887, 4.469936)

In [29]:
cur.close()
conn.close()

ProgrammingError: Cannot operate on a closed database.



## Option 2: Write insert statements in the code


In [ ]:
import sqlite3

In [ ]:
sqlite_filename = '/Users/muhammadwaliji/Desktop/project_2/who_country_code_mhw.sqlite'

In [ ]:
conn = sqlite3.connect(sqlite_filename)
cur = conn.cursor()


In [ ]:
cur.execute("DROP TABLE IF EXISTS long_lat;")

In [ ]:
cur.execute("""
CREATE TABLE long_lat (
    id INTEGER PRIMARY KEY,
    country VARCHAR(255),
    country_code VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT
)
""")

In [ ]:
# closing cursor and connection commits the changes you have made to the database. If you do not close
# the changes may not have saved in the sqlite file. 
# Another reason to close also is that sqlite can only run in one place at one time so you have to close the 
# connection here in order to run it on the terminal
conn.commit()
cur.close()
conn.close()

In [ ]:
conn = sqlite3.connect(sqlite_filename)
cur = conn.cursor()

In [ ]:
# for idx, row in who_and_others.iterrows():
#     cur.execute(f"""
#         INSERT INTO long_lat(id, country, country_code, latitude, longitude) VALUES 
#         ({idx},'{row.country}','{row.country_code}', {row.latitude}, {row.longitude})
#     """)
# conn.commit()

In [ ]:
for idx, row in who_and_others.iterrows():
    cur.execute(f"""
        INSERT INTO long_lat(id, country, country_code, latitude, longitude) VALUES 
        (?, ?, ?, ?, ?)
    """,
               [idx,
               row.country,
               row.country_code,
               row.latitude,
               row.longitude])
conn.commit()

In [ ]:
cur.close()
conn.close()

In [ ]:
pd.options.display.max_rows = 999

## SQL ALCHEMY STARTS HERE

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.orm import Session

from sqlalchemy.ext.automap import automap_base

In [ ]:
engine=create_engine("sqlite:///Users/muhammadwaliji/Desktop/project_2/who_country_code.sqlite")
Base = automap_base()


In [ ]:
session = Session(engine)

In [ ]:
Base.prepare(engine, reflect=True)

In [ ]:
class who_country_code(Base):
    __tablename__ = 'long_lat'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    country_code = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    

In [ ]:
Base.metadata.tables

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
session = Session(engine)

In [ ]:
# URLS used
# 1. table of countries and long lats
# https://developers.google.com/public-data/docs/canonical/countries_csv
# 2. Make data frame with 3 and 2 letter country code
# https://www.worldatlas.com/aatlas/ctycodes.htm
